# Dual CRISPR Screen Analysis
# Count Combination
Amanda Birmingham, CCBB, UCSD (abirmingham@ucsd.edu)

## Instructions

To run this notebook reproducibly, follow these steps:
1. Click **Kernel** > **Restart & Clear Output**
2. When prompted, click the red **Restart & clear all outputs** button
3. Fill in the values for your analysis for each of the variables in the [Input Parameters](#input-parameters) section
4. Click **Cell** > **Run All**

<a name = "input-parameters"></a>

## Input Parameters

In [1]:
g_timestamp = '20160713235254'
g_dataset_name = '20160706_HeLa_A549_MV4'
g_count_alg_name = '19mer_1mm_py'
g_fastq_counts_dir = '/data/interim/20160706_D00611_0304_BHVVJ3BCXX'
g_fastq_counts_run_prefix = '20160706_HeLa_A549_MV4_20160713235254'
g_collapsed_counts_dir = '/data/processed/20160706_HeLa_A549_MV4_19mer_1mm_py_20160713235254'
g_collapsed_counts_run_prefix = '20160706_HeLa_A549_MV4_19mer_1mm_py_20160713235254'
g_combined_counts_dir = '/data/processed/20160706_HeLa_A549_MV4_19mer_1mm_py_20160713235254'
g_combined_counts_run_prefix = '20160706_HeLa_A549_MV4_19mer_1mm_py_20160713235254'
g_code_location = '/home/ec2-user/jupyter-genomics/src/crispr'

## CCBB Library Imports

In [2]:
import sys
sys.path.append(g_code_location)

## Automated Set-Up

In [3]:
# %load -s describe_var_list /Users/Birmingham/Repositories/ccbb_tickets/20160210_mali_crispr/src/python/ccbbucsd/utilities/analysis_run_prefixes.py
def describe_var_list(input_var_name_list):
    description_list =  ["{0}: {1}\n".format(name, eval(name)) for name in input_var_name_list]
    return "".join(description_list)


In [4]:
from ccbbucsd.utilities.analysis_run_prefixes import check_or_set, get_run_prefix, get_timestamp
g_timestamp = check_or_set(g_timestamp, get_timestamp())
g_collapsed_counts_dir = check_or_set(g_collapsed_counts_dir, g_fastq_counts_dir)
g_collapsed_counts_run_prefix = check_or_set(g_collapsed_counts_run_prefix, 
                                             get_run_prefix(g_dataset_name, g_count_alg_name, g_timestamp))
g_combined_counts_dir = check_or_set(g_combined_counts_dir, g_collapsed_counts_dir)
g_combined_counts_run_prefix = check_or_set(g_combined_counts_run_prefix, g_collapsed_counts_run_prefix)
print(describe_var_list(['g_timestamp','g_collapsed_counts_dir','g_collapsed_counts_run_prefix', 
                         'g_combined_counts_dir', 'g_combined_counts_run_prefix']))

g_timestamp: 20160713235254
g_collapsed_counts_dir: /data/processed/20160706_HeLa_A549_MV4_19mer_1mm_py_20160713235254
g_collapsed_counts_run_prefix: 20160706_HeLa_A549_MV4_19mer_1mm_py_20160713235254
g_combined_counts_dir: /data/processed/20160706_HeLa_A549_MV4_19mer_1mm_py_20160713235254
g_combined_counts_run_prefix: 20160706_HeLa_A549_MV4_19mer_1mm_py_20160713235254



In [5]:
from ccbbucsd.utilities.files_and_paths import verify_or_make_dir
verify_or_make_dir(g_collapsed_counts_dir)
verify_or_make_dir(g_combined_counts_dir)

## Count Combination Functions

In [6]:
# %load -s get_counts_file_suffix /Users/Birmingham/Repositories/ccbb_tickets/20160210_mali_crispr/src/python/ccbbucsd/malicrispr/construct_counter.py
def get_counts_file_suffix():
    return "counts.txt"


In [7]:
# %load /Users/Birmingham/Repositories/ccbb_tickets/20160210_mali_crispr/src/python/ccbbucsd/malicrispr/count_combination.py
# ccbb libraries
from ccbbucsd.utilities.analysis_run_prefixes import strip_run_prefix
from ccbbucsd.utilities.files_and_paths import build_multipart_fp, group_files, get_filepaths_by_prefix_and_suffix

# project-specific libraries
from ccbbucsd.malicrispr.count_files_and_dataframes import get_counts_df

__author__ = "Amanda Birmingham"
__maintainer__ = "Amanda Birmingham"
__email__ = "abirmingham@ucsd.edu"
__status__ = "prototype"


def get_collapsed_counts_file_suffix():
    return "collapsed.txt"


def get_combined_counts_file_suffix():
    return "counts_combined.txt"


def group_lane_and_set_files(filepaths):
    # NB: this regex assumes read designator has *already* been removed
    # and replaced with _ as done by group_read_pairs
    return group_files(filepaths, "_L\d\d\d_\d\d\d", "")


def combine_count_files(counts_fp_for_dataset, run_prefix):
    combined_df = None
    
    for curr_counts_fp in counts_fp_for_dataset:
        count_header, curr_counts_df = get_counts_df(curr_counts_fp, run_prefix)
        
        if combined_df is None:
            combined_df = curr_counts_df
        else:
            combined_df[count_header] = curr_counts_df[count_header]
    
    return combined_df


def write_collapsed_count_files(input_dir, output_dir, curr_run_prefix, counts_run_prefix, counts_suffix, counts_collapsed_file_suffix):
    counts_fps_for_dataset = get_filepaths_by_prefix_and_suffix(input_dir, counts_run_prefix, counts_suffix)
    fps_by_sample = group_lane_and_set_files(counts_fps_for_dataset)
    
    for curr_sample, curr_fps in fps_by_sample.items():
        stripped_sample = strip_run_prefix(curr_sample, counts_run_prefix)
        output_fp = build_multipart_fp(output_dir, [curr_run_prefix, stripped_sample, counts_collapsed_file_suffix]) 
        combined_df = None        
        
        for curr_fp in curr_fps:
            count_header, curr_counts_df = get_counts_df(curr_fp, counts_run_prefix)
        
            if combined_df is None:
                combined_df = curr_counts_df
                combined_df.rename(columns = {count_header:stripped_sample}, inplace = True) 
            else:
                combined_df[stripped_sample] = combined_df[stripped_sample] + curr_counts_df[count_header]
    
        combined_df.to_csv(output_fp, sep="\t", index=False)    


def write_combined_count_file(input_dir, output_dir, curr_run_prefix, counts_run_prefix, counts_suffix, combined_suffix):
    output_fp = build_multipart_fp(output_dir, [curr_run_prefix, combined_suffix])
    counts_fps_for_run = get_filepaths_by_prefix_and_suffix(input_dir, counts_run_prefix, counts_suffix)
    combined_df = combine_count_files(counts_fps_for_run, curr_run_prefix)
    combined_df.to_csv(output_fp, sep="\t", index=False)

## Input Count Filenames

In [8]:
from ccbbucsd.utilities.files_and_paths import summarize_filenames_for_prefix_and_suffix
print(summarize_filenames_for_prefix_and_suffix(g_fastq_counts_dir, g_fastq_counts_run_prefix, get_counts_file_suffix()))

A549_MV4_d14_1_S3_L001_001_trimmed53_len_filtered_20160706_HeLa_A549_MV4_20160713235254_counts.txt
A549_MV4_d14_1_S3_L002_001_trimmed53_len_filtered_20160706_HeLa_A549_MV4_20160713235254_counts.txt
A549_MV4_d14_2_S4_L001_001_trimmed53_len_filtered_20160706_HeLa_A549_MV4_20160713235254_counts.txt
A549_MV4_d14_2_S4_L002_001_trimmed53_len_filtered_20160706_HeLa_A549_MV4_20160713235254_counts.txt
A549_MV4_d20_1_S5_L001_001_trimmed53_len_filtered_20160706_HeLa_A549_MV4_20160713235254_counts.txt
A549_MV4_d20_1_S5_L002_001_trimmed53_len_filtered_20160706_HeLa_A549_MV4_20160713235254_counts.txt
A549_MV4_d20_2_S6_L001_001_trimmed53_len_filtered_20160706_HeLa_A549_MV4_20160713235254_counts.txt
A549_MV4_d20_2_S6_L002_001_trimmed53_len_filtered_20160706_HeLa_A549_MV4_20160713235254_counts.txt
A549_MV4_d28_1_S7_L001_001_trimmed53_len_filtered_20160706_HeLa_A549_MV4_20160713235254_counts.txt
A549_MV4_d28_1_S7_L002_001_trimmed53_len_filtered_20160706_HeLa_A549_MV4_20160713235254_counts.txt
A549_MV4_d

## Count Combination Execution

In [9]:
write_collapsed_count_files(g_fastq_counts_dir, g_collapsed_counts_dir, g_collapsed_counts_run_prefix, 
                            g_fastq_counts_run_prefix, get_counts_file_suffix(), get_collapsed_counts_file_suffix())

In [10]:
write_combined_count_file(g_collapsed_counts_dir, g_combined_counts_dir, g_collapsed_counts_run_prefix, 
                          g_combined_counts_run_prefix, get_collapsed_counts_file_suffix(), 
                          get_combined_counts_file_suffix())